In [1]:
import requests
import pandas as pd
import altair as alt

## Setup API

In [2]:
class Libraries:
    def __init__(self, api_file_path='./local/api.txt'):
        
        self.api_file_path = api_file_path
        
        self.api_key = None
        self.payload = None
        self.url = None
        self.r  = None
        self.json = None
        self.json_flat = None
        
        self.load_api_key()
        
        return
    
    def load_api_key(self):
        with open(self.api_file_path, 'r') as file:
            self.api_key = file.read()
        return
    
    def create_payload(self, paramenters=None):
        self.payload = dict()
        self.payload.update({'api_key': self.api_key})
        return
    
    def get_response(self):
        self.r = requests.get(self.url, params=self.payload)
        return
    
    def get_package(self, repository='Pypi', package='requests'):
        self.url = 'https://libraries.io/api/{}/{}'.format(repository, package)
        self.get_response()
        self.json = self.r.json()
        self.json_flat = self.r.json()
        del self.json_flat['versions']
        del self.json_flat['normalized_licenses']
        del self.json_flat['keywords']
        del self.json_flat['latest_stable_release']
        
        return self.json_flat
    
lib = Libraries()
data = lib.get_package(repository='Pypi', package='numpy')


# Create csv of data from packages

In [3]:
packages = [
    ['Pypi', 'numpy'],
    ['Pypi', 'requests'],
    ['Pypi', 'plotly'],
    ['Pypi', 'scipy'],
    ['Pypi', 'altair']
]
lib = Libraries()

def make_dataframe(packages, lib):
    package_dict = dict()
    i = 0
    for package in packages:
        package_dict[i] = lib.get_package(repository=package[0], package=package[1])
        i =i+1
    return pd.DataFrame.from_dict(package_dict, orient='index').reindex()
        
df =make_dataframe(packages, lib)
df.to_csv('package-data.csv')

In [4]:
df

,dependent_repos_count,dependents_count,deprecation_reason,description,forks,homepage,language,latest_download_url,latest_release_number,latest_release_published_at,...,latest_stable_release_published_at,license_normalized,licenses,name,package_manager_url,platform,rank,repository_url,stars,status
0,84861,4992,None,NumPy is the fundamental package for array com...,4732,https://www.numpy.org,C,None,1.19.1,2020-07-21T20:54:49.000Z,...,2020-07-21T20:54:49.000Z,True,BSD,numpy,https://pypi.org/project/numpy/,Pypi,29,https://github.com/numpy/numpy,14434,
1,152228,14039,None,Python HTTP for Humans.,7803,https://requests.readthedocs.io,Python,None,2.24.0,2020-06-17T16:30:08.000Z,...,2020-06-17T16:30:08.000Z,True,Apache 2.0,requests,https://pypi.org/project/requests/,Pypi,31,https://github.com/psf/requests,43100,
2,2491,89,None,"An open-source, interactive data visualization...",0,https://plotly.com/python/,None,None,4.9.0,2020-07-16T12:46:37.000Z,...,2020-07-16T12:46:37.000Z,False,MIT,plotly,https://pypi.org/project/plotly/,Pypi,14,None,0,None
3,43431,993,None,SciPy: Scientific Library for Python,3345,https://www.scipy.org,Python,None,1.5.2,2020-07-24T01:48:43.000Z,...,2020-07-24T01:48:43.000Z,True,BSD,scipy,https://pypi.org/project/scipy/,Pypi,26,https://github.com/scipy/scipy,7342,
4,185,13,None,Altair: A declarative statistical visualizatio...,19,http://altair-viz.github.io,Python,None,4.1.0,2020-04-01T13:23:10.000Z,...,2020-04-01T13:23:10.000Z,True,BSD 3-clause,altair,https://pypi.org/project/altair/,Pypi,14,https://github.com/ellisonbg/altair,233,None


# Create vega-lite visualization using csv

In [5]:
url = 'https://raw.githubusercontent.com/library-usage/library-sync/master/package-data.csv'

chart = alt.Chart(url, width=400, height=400).mark_point().encode(
    x='rank:Q',
    y='stars:Q',
    color='name:N',
    tooltip='name:N',
).interactive()
chart

alt.Chart(...)

In [6]:
chart.save('stars.json')